# Impermanent loss notes

In [1]:
# Simplified swap without fees
import math

# LIQUIDITY
# given some amount of an asset and pair reserves, returns an equivalent amount of the other asset
# used when adding liquidity
def quote(amountA, reserveA, reserveB):
    assert(amountA > 0)
    assert(reserveA > 0 and reserveB > 0)
    amountB = (amountA * reserveB) / reserveA
    return amountB

# SWAP
# simplified version of above
def get_amount_delta(amount0_delta, reserve0, reserve1):
    dx = amount0_delta
    x = reserve0
    y = reserve1
    return (-dx*y)/(x+dx)

# Simple ERC20 token
class SimpleToken:
    def __init__(self):
        self.balance_of = {}
        self.total_supply = 0
    def _mint(self, to, value):
        self.total_supply += value
        self.balance_of[to] = self.balance_of.get(to, 0) + value
    def _burn(self, from_, value):
        self.total_supply -= value
        self.balance_of[from_] = self.balance_of.get(from_, 0) - value
    
class SimpleSwap(SimpleToken):
    def __init__(self):
        super().__init__();
        self.reserve0 = 0
        self.reserve1 = 0
    def k(self):
        return self.reserve0 * self.reserve1
    def liquidity(self):
        return math.sqrt(self.k())
    
    def mint(self, to, amount0 = None, amount1 = None):
        assert(not (amount0 is None and amount1 is None));
        if amount0 is None:
            amount0 = quote(amount1, self.reserve1, self.reserve0)
        if amount1 is None:
            amount1 = quote(amount0, self.reserve0, self.reserve1)
            
        if self.total_supply == 0:
            # first mint
            liquidity = math.sqrt(amount0 * amount1)
            # omitted min. liquidity code for simplicity
        else :
            l1 = amount0/self.reserve0 * self.total_supply
            l2 = amount1/self.reserve1 * self.total_supply
            # ensures that amounts are added in same proportion otherwise the smallest percent wins?
            liquidity = Math.min(l1, l2)
        self.reserve0 += amount0
        self.reserve1 += amount1
        self._mint(to, liquidity)
        
    def swap(self, amount0_delta):
        # balance represented token reserve after a successful swap
        amount1_delta = get_amount_delta(amount0_delta, self.reserve0, self.reserve1)
        reserve0_new = self.reserve0 + amount0_delta
        reserve1_new = self.reserve1 + amount1_delta
        assert (reserve0_new * reserve1_new == self.k()), 'k invariant is not preserved'
        self.reserve0 = reserve0_new
        self.reserve1 = reserve1_new
    def price0(self):
        return self.reserve1/self.reserve0
    def price1(self):
        return self.reserve0/self.reserve1
    def pprint(self):
        print("k = {}".format(self.k()))
        print("lp_supply = {}".format(self.total_supply))
        print("reserve0 = {}".format(self.reserve0))
        print("reserve1 = {}".format(self.reserve1))
        print("LPs = {}".format(self.balance_of))
        print("1 token0 = {} token1".format(self.price0()))
        print("1 token1 = {} token0".format(self.price1()))
    

In [2]:
swap = SimpleSwap()
swap.mint("0xbob", 1000, 20000)
swap.pprint()

k = 20000000
lp_supply = 4472.13595499958
reserve0 = 1000
reserve1 = 20000
LPs = {'0xbob': 4472.13595499958}
1 token0 = 20.0 token1
1 token1 = 0.05 token0


In [3]:
swap.swap(100)
swap.pprint()

k = 20000000.0
lp_supply = 4472.13595499958
reserve0 = 1100
reserve1 = 18181.81818181818
LPs = {'0xbob': 4472.13595499958}
1 token0 = 16.52892561983471 token1
1 token1 = 0.060500000000000005 token0


In [4]:
swap.swap(-100)
swap.pprint()

k = 20000000.0
lp_supply = 4472.13595499958
reserve0 = 1000
reserve1 = 20000.0
LPs = {'0xbob': 4472.13595499958}
1 token0 = 20.0 token1
1 token1 = 0.05 token0


In [5]:
def delta_x_for_price_x(desired_price_x, reserve0, reserve1):
    p = desired_price_x
    x = reserve0
    y = reserve1
    dx = math.sqrt(p*x*y) - x
    return dx
delta_x_for_price_x(16.52, 1000, 20000)

17176.908428002822